Just add the zip file path properly. To run the code.

# 3D CNN

### Data Loading

In [1]:
!unzip '/content/Cardiac-Video-Sequence.zip'

Archive:  /content/checkpoints.zip
   creating: checkpoints/my_3d_cnn_model/
 extracting: checkpoints/my_3d_cnn_model/fingerprint.pb  
  inflating: checkpoints/my_3d_cnn_model/keras_metadata.pb  
  inflating: checkpoints/my_3d_cnn_model/saved_model.pb  
   creating: checkpoints/my_3d_cnn_model/variables/
  inflating: checkpoints/my_3d_cnn_model/variables/variables.data-00000-of-00001  
  inflating: checkpoints/my_3d_cnn_model/variables/variables.index  
  inflating: checkpoints/my_vit_dense_model.h5  
  inflating: checkpoints/my_vit_lstm_model.h5  
Archive:  /content/Cardiac-Video-Sequence.zip
   creating: Cardiac-Video-Sequence/
  inflating: __MACOSX/._Cardiac-Video-Sequence  
  inflating: Cardiac-Video-Sequence/2023-11-15-cine-myo-masks-and-TOS.npy  
  inflating: __MACOSX/Cardiac-Video-Sequence/._2023-11-15-cine-myo-masks-and-TOS.npy  
  inflating: Cardiac-Video-Sequence/README.txt  
  inflating: __MACOSX/Cardiac-Video-Sequence/._README.txt  


In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from numpy import pad

### Data preprocessing  

In [2]:
import numpy as np

data = np.load('/content/Cardiac-Video-Sequence/2023-11-15-cine-myo-masks-and-TOS.npy', allow_pickle=True)

mask_volume = []
TOS_volume = []
time_stamps = []

for i in range(128):
    mask_volume.append(data[i]['cine_lv_myo_masks_cropped'])   # Get the myocardium mask of slice 0. It should be a (H, W, n_frames) volume
    TOS_volume.append(data[i]['TOS'])  # Get the TOS curve of slice 0. It should be a (126, n_frames) 2D array

print('Data:', mask_volume[0].shape)
print('Ground Truth', TOS_volume[0].shape)

mask_volume = []
TOS_volume = []

# Extract data
for i in range(len(data)):
    mask_volume.append(data[i]['cine_lv_myo_masks_cropped'])  # Myocardium masks
    TOS_volume.append(data[i]['TOS'])  # TOS curves

# Determine the maximum dimensions
max_height = max(array.shape[0] for array in mask_volume)
max_width = max(array.shape[1] for array in mask_volume)
max_frames = max(array.shape[2] for array in mask_volume)

# Function to pad an array to the maximum size
def pad_array(array, max_height, max_width, max_frames):
    pad_height = max_height - array.shape[0]
    pad_width = max_width - array.shape[1]
    pad_frames = max_frames - array.shape[2]
    return pad(array, ((0, pad_height), (0, pad_width), (0, pad_frames)), 'constant', constant_values=0)

# Apply padding to each array
mask_volume_padded = [pad_array(array, max_height, max_width, max_frames) for array in mask_volume]

# Convert lists to numpy arrays
mask_volume = np.array(mask_volume_padded)
TOS_volume = np.array(TOS_volume)

# Normalizing the mask volumes
mask_volume = mask_volume / 255.0

# Splitting the dataset into training and testing sets
mask_volume_train, mask_volume_test, TOS_volume_train, TOS_volume_test = train_test_split(mask_volume, TOS_volume, test_size=0.2, random_state=42)

print("Training Data Shape:", mask_volume_train.shape)
print("Testing Data Shape:", mask_volume_test.shape)
print("Training Ground Truth Shape:", TOS_volume_train.shape)
print("Testing Ground Truth Shape:", TOS_volume_test.shape)


# Convert training and testing labels to integer values
TOS_volume_train_int = np.round(TOS_volume_train).astype(int)
TOS_volume_test_int = np.round(TOS_volume_test).astype(int)

print("Training Ground Truth Shape:", TOS_volume_train_int.shape)
print("Testing Ground Truth Shape:", TOS_volume_test_int.shape)

Data: (80, 80, 25)
Ground Truth (126,)
Training Data Shape: (102, 80, 80, 25)
Testing Data Shape: (26, 80, 80, 25)
Training Ground Truth Shape: (102, 126)
Testing Ground Truth Shape: (26, 126)
Training Ground Truth Shape: (102, 126)
Testing Ground Truth Shape: (26, 126)


### Load and check the model on the test dataset on 3D CNN

In [4]:
# Load the saved model
model = tf.keras.models.load_model('/content/checkpoints/my_3d_cnn_model')

# Evaluate the model on the test data
loss, mean_absolute_error = model.evaluate(mask_volume_test, TOS_volume_test)

print("Test Loss:", loss)
print("Test Mean Absolute Error:", mean_absolute_error)

1/1 [==============================] - 0s 253ms/step - loss: 198.5519 - mean_absolute_error: 8.4136
Test Loss: 198.55194091796875
Test Mean Absolute Error: 8.413644790649414


Please note we have reported the test scores MAE value from the main 3D_CNN.ipynb file in the report. ANd not this. MAE = 8.5879

----
----

# ViT + Dense

### Data Loading

In [3]:
!unzip '/content/Cardiac-Video-Sequence.zip'

Archive:  /content/Cardiac-Video-Sequence.zip
   creating: Cardiac-Video-Sequence/
  inflating: __MACOSX/._Cardiac-Video-Sequence  
  inflating: Cardiac-Video-Sequence/2023-11-15-cine-myo-masks-and-TOS.npy  
  inflating: __MACOSX/Cardiac-Video-Sequence/._2023-11-15-cine-myo-masks-and-TOS.npy  
  inflating: Cardiac-Video-Sequence/README.txt  
  inflating: __MACOSX/Cardiac-Video-Sequence/._README.txt  


In [5]:
import numpy as np
from numpy import pad
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub

### Data preprocessing  

In [6]:
data = np.load('/content/Cardiac-Video-Sequence/2023-11-15-cine-myo-masks-and-TOS.npy', allow_pickle=True)

mask_volume = []
TOS_volume = []
time_stamps = []

for i in range(128):
    mask_volume.append(data[i]['cine_lv_myo_masks_cropped'])   # Get the myocardium mask of slice 0. It should be a (H, W, n_frames) volume
    TOS_volume.append(data[i]['TOS'])  # Get the TOS curve of slice 0. It should be a (126, n_frames) 2D array

print('Data:', mask_volume[0].shape)
print('Ground Truth', TOS_volume[0].shape)

mask_volume = []
TOS_volume = []

for i in range(128):
    mask = data[i]['cine_lv_myo_masks_cropped']   # Shape: (H, W, n_frames)
    TOS = data[i]['TOS']                          # Shape: (126,)

    # Example preprocessing step: averaging across the time dimension
    mask_avg = np.mean(mask, axis=2)              # Shape: (H, W)

    mask_volume.append(mask_avg)
    TOS_volume.append(TOS)

# Normalize the data
mask_volume = np.array(mask_volume)
mask_volume = (mask_volume - np.min(mask_volume)) / (np.max(mask_volume) - np.min(mask_volume))

# Convert TOS_volume to a numpy array for consistency
TOS_volume = np.array(TOS_volume)

# Further steps include splitting the data into training, validation, and test sets,
# and then batching the data for training.

print('Data:', mask_volume.shape)
print('Ground Truth', TOS_volume.shape)

mask_volume = []
TOS_volume = []

# Extract data
for i in range(len(data)):
    mask_volume.append(data[i]['cine_lv_myo_masks_cropped'])  # Myocardium masks
    TOS_volume.append(data[i]['TOS'])  # TOS curves

# Determine the maximum dimensions
max_height = max(array.shape[0] for array in mask_volume)
max_width = max(array.shape[1] for array in mask_volume)
max_frames = max(array.shape[2] for array in mask_volume)

# Function to pad an array to the maximum size
def pad_array(array, max_height, max_width, max_frames):
    pad_height = max_height - array.shape[0]
    pad_width = max_width - array.shape[1]
    pad_frames = max_frames - array.shape[2]
    return pad(array, ((0, pad_height), (0, pad_width), (0, pad_frames)), 'constant', constant_values=0)

# Apply padding to each array
mask_volume_padded = [pad_array(array, max_height, max_width, max_frames) for array in mask_volume]

# Convert lists to numpy arrays
mask_volume = np.array(mask_volume_padded)
TOS_volume = np.array(TOS_volume)

# Normalizing the mask volumes
mask_volume = mask_volume / 255.0

# Splitting the dataset into training and testing sets
mask_volume_train, mask_volume_test, TOS_volume_train, TOS_volume_test = train_test_split(mask_volume, TOS_volume, test_size=0.2, random_state=42)

print("Training Data Shape:", mask_volume_train.shape)
print("Testing Data Shape:", mask_volume_test.shape)
print("Training Ground Truth Shape:", TOS_volume_train.shape)
print("Testing Ground Truth Shape:", TOS_volume_test.shape)

# Splitting data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(mask_volume, TOS_volume, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)
print(X_test.shape,y_test.shape)

# Convert training and testing labels to integer values
TOS_volume_train_int = np.round(TOS_volume_train).astype(int)
TOS_volume_test_int = np.round(TOS_volume_test).astype(int)

print("Training Ground Truth Shape:", TOS_volume_train_int.shape)
print("Testing Ground Truth Shape:", TOS_volume_test_int.shape)



Data: (80, 80, 25)
Ground Truth (126,)
Data: (128, 80, 80)
Ground Truth (128, 126)
Training Data Shape: (102, 80, 80, 25)
Testing Data Shape: (26, 80, 80, 25)
Training Ground Truth Shape: (102, 126)
Testing Ground Truth Shape: (26, 126)
(102, 80, 80, 25) (102, 126)
(13, 80, 80, 25) (13, 126)
(13, 80, 80, 25) (13, 126)
Training Ground Truth Shape: (102, 126)
Testing Ground Truth Shape: (26, 126)


### Load and check the model on the test dataset on ViT+ Dense

In [7]:
# Load the saved model
loaded_model = tf.keras.models.load_model(
    '/content/checkpoints/my_vit_dense_model.h5',
    custom_objects={'KerasLayer': hub.KerasLayer}
)


# Evaluate the model
loss, mean_absolute_error = loaded_model.evaluate(mask_volume_test, TOS_volume_test_int)
print("Test Loss:", loss)
print("Test Mean Absolute Error:", mean_absolute_error)

1/1 [==============================] - 19s 19s/step - loss: 200.8010 - mean_absolute_error: 8.2325
Test Loss: 200.80096435546875
Test Mean Absolute Error: 8.232523918151855


---
---

# ViT + LSTM

### Data Loading

In [14]:
!unzip '/content/Cardiac-Video-Sequence.zip'

Archive:  /content/Cardiac-Video-Sequence.zip
replace __MACOSX/._Cardiac-Video-Sequence? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._Cardiac-Video-Sequence  
replace Cardiac-Video-Sequence/2023-11-15-cine-myo-masks-and-TOS.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Cardiac-Video-Sequence/2023-11-15-cine-myo-masks-and-TOS.npy  
  inflating: __MACOSX/Cardiac-Video-Sequence/._2023-11-15-cine-myo-masks-and-TOS.npy  
  inflating: Cardiac-Video-Sequence/README.txt  
  inflating: __MACOSX/Cardiac-Video-Sequence/._README.txt  


In [8]:
import numpy as np
from numpy import pad
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub

### Data preprocessing  

In [10]:
# Load data
data = np.load('/content/Cardiac-Video-Sequence/2023-11-15-cine-myo-masks-and-TOS.npy', allow_pickle=True)

mask_volume = []
TOS_volume = []
time_stamps = []

for i in range(128):
    mask_volume.append(data[i]['cine_lv_myo_masks_cropped'])   # Get the myocardium mask of slice 0. It should be a (H, W, n_frames) volume
    TOS_volume.append(data[i]['TOS'])  # Get the TOS curve of slice 0. It should be a (126, n_frames) 2D array

print('Data:', mask_volume[0].shape)
print('Ground Truth', TOS_volume[0].shape)



mask_volume = []
TOS_volume = []

for i in range(128):
    mask = data[i]['cine_lv_myo_masks_cropped']   # Shape: (H, W, n_frames)
    TOS = data[i]['TOS']                          # Shape: (126,)

    # Example preprocessing step: averaging across the time dimension
    mask_avg = np.mean(mask, axis=2)              # Shape: (H, W)

    mask_volume.append(mask_avg)
    TOS_volume.append(TOS)

# Normalize the data
mask_volume = np.array(mask_volume)
mask_volume = (mask_volume - np.min(mask_volume)) / (np.max(mask_volume) - np.min(mask_volume))

# Convert TOS_volume to a numpy array for consistency
TOS_volume = np.array(TOS_volume)

# Further steps include splitting the data into training, validation, and test sets,
# and then batching the data for training.

print('Data:', mask_volume.shape)
print('Ground Truth', TOS_volume.shape)

mask_volume = []
TOS_volume = []

# Extract data
for i in range(len(data)):
    mask_volume.append(data[i]['cine_lv_myo_masks_cropped'])  # Myocardium masks
    TOS_volume.append(data[i]['TOS'])  # TOS curves

# Determine the maximum dimensions
max_height = max(array.shape[0] for array in mask_volume)
max_width = max(array.shape[1] for array in mask_volume)
max_frames = max(array.shape[2] for array in mask_volume)

# Function to pad an array to the maximum size
def pad_array(array, max_height, max_width, max_frames):
    pad_height = max_height - array.shape[0]
    pad_width = max_width - array.shape[1]
    pad_frames = max_frames - array.shape[2]
    return pad(array, ((0, pad_height), (0, pad_width), (0, pad_frames)), 'constant', constant_values=0)

# Apply padding to each array
mask_volume_padded = [pad_array(array, max_height, max_width, max_frames) for array in mask_volume]

# Convert lists to numpy arrays
mask_volume = np.array(mask_volume_padded)
TOS_volume = np.array(TOS_volume)

# Normalizing the mask volumes
mask_volume = mask_volume / 255.0

# Splitting the dataset into training and testing sets
mask_volume_train, mask_volume_test, TOS_volume_train, TOS_volume_test = train_test_split(mask_volume, TOS_volume, test_size=0.2, random_state=42)

print("Training Data Shape:", mask_volume_train.shape)
print("Testing Data Shape:", mask_volume_test.shape)
print("Training Ground Truth Shape:", TOS_volume_train.shape)
print("Testing Ground Truth Shape:", TOS_volume_test.shape)

# Splitting data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(mask_volume, TOS_volume, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)
print(X_test.shape,y_test.shape)

# Convert training and testing labels to integer values
TOS_volume_train_int = np.round(TOS_volume_train).astype(int)
TOS_volume_test_int = np.round(TOS_volume_test).astype(int)

print("Training Ground Truth Shape:", TOS_volume_train_int.shape)
print("Testing Ground Truth Shape:", TOS_volume_test_int.shape)



Data: (80, 80, 25)
Ground Truth (126,)
Data: (128, 80, 80)
Ground Truth (128, 126)
Training Data Shape: (102, 80, 80, 25)
Testing Data Shape: (26, 80, 80, 25)
Training Ground Truth Shape: (102, 126)
Testing Ground Truth Shape: (26, 126)
(102, 80, 80, 25) (102, 126)
(13, 80, 80, 25) (13, 126)
(13, 80, 80, 25) (13, 126)
Training Ground Truth Shape: (102, 126)
Testing Ground Truth Shape: (26, 126)


### Load and check the model on the test dataset on ViT+lstm

In [11]:
# Load the saved model
loaded_model = tf.keras.models.load_model(
    '/content/checkpoints/my_vit_lstm_model.h5',
    custom_objects={'KerasLayer': hub.KerasLayer}
)


# Evaluate the model
loss, mean_absolute_error = loaded_model.evaluate(mask_volume_test, TOS_volume_test_int)
print("Test Loss:", loss)
print("Test Mean Absolute Error:", mean_absolute_error)

1/1 [==============================] - 9s 9s/step - loss: 199.6137 - mean_absolute_error: 8.1138
Test Loss: 199.61366271972656
Test Mean Absolute Error: 8.113755226135254


----

Thanks!

Please note we have reported the test scores MAE value from the main Traning ipynb files

Reach me out if you encounter any problem!
rugvedm12@gmail.com
😄

---